In [34]:
import requests
import pandas as pd
from selenium import webdriver
from scrapy.http import TextResponse
import seaborn as sns
import getpass
import re
import time
from selenium.webdriver.chrome.options import Options
import pprint

###

In [3]:
chrome_options = Options()
# chrome_options.add_argument('headless')
chrome_options.add_argument("--window-size=1600,1080")

In [4]:
# 여기서부터 시작`

In [5]:
login_url = "https://www.instagram.com/accounts/login/?source=auth_switcher"

driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get(login_url)

/Users/song-yeejun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# driver.find_element_by_css_selector('#slfErrorAlert').text

In [11]:
# element의 렌더링을 기다려야 하는 경우, 렌더링 될때까지 재귀적으로 함수 실행
def check_response(webdriver, selector, func, start_time):
    try:
        if func == 'login':
            result = webdriver.find_element_by_css_selector(selector)

        elif func == 'input_keyword':
            if time.time() - start_time > 10:
                print(
                    "=========== no response in function : {} ===========".format(func))
                return False

            result = webdriver.find_elements_by_css_selector(selector)
            while len(result) == 0:
                return check_response(webdriver, selector, func, start_time)

        elif func == 'initial_crawling':
            if time.time() - start_time > 10:
                print(
                    "=========== no response in function : {} ===========".format(func))
                return False

            result = webdriver.find_elements_by_css_selector(selector)
            while len(result) != 8:
                return check_response(webdriver, selector, func, start_time)

        return result

    except Exception:
        # func = 'login'인 경우 element를 찾지 못하면 에러처리로 들어온다

        if time.time() - start_time > 5:
            print(
                "=========== no response in function : {} ===========".format(func))
            return False

        else:
            return check_response(webdriver, selector, func, start_time)

In [12]:
# 로그인 함수 (seleniun webdriver)


def login():
    my_id = input('id를 입력하세요: ')
    my_password = getpass.getpass('password를 입력하세요: ')

    driver.find_element_by_css_selector(
        '._2hvTZ.pexuQ.zyHYP[type=text]').send_keys(my_id)
    driver.find_element_by_css_selector(
        '._2hvTZ.pexuQ.zyHYP[type=password]').send_keys(my_password)
    driver.find_element_by_css_selector(
        '.sqdOP.L3NKy.y3zKF[type=submit]').click()

    # 알림설정 하라는 modal창이 뜨는경우 '나중에 하기'를 클릭하는 코드
    start_time = time.time()
    alert_modal = check_response(
        driver, 'body > div.RnEpo.Yx5HN > div > div', 'login', start_time)
    if alert_modal == False:
        print('알림설정창 안뜸')
        return

    driver.find_element_by_css_selector(
        'body > div.RnEpo.Yx5HN > div > div > div.mt3GC > button.aOOlW.HoLwm').click()

In [18]:
# 검색어를 입력하고 게시물이 가장 많은 검색결과를 클릭 (seleniun webdriver)


def input_keyword(word):
    keyword = word

    driver.find_element_by_css_selector(
        '#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > input').send_keys(keyword)

    # 키워드 입력하고, 키워드에 대한 검색 리스트가 뜨는데 걸리는 시간만큼 기다려준다
    start_time = time.time()
    search_list = check_response(
        driver, '#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > div:nth-child(4) > div.drKGC > div > a', 'input_keyword', start_time)
    if search_list == False:
        print('================ input_keyword rendering error ================')
        driver.quit()

    # 서칭된 a 엘리먼트들 중에 게시물 갯수가 가장 많은 a 엘리먼트를 뽑는다
    number_list = []

    for element in search_list:
        splited_element = element.text.split('게시물')
        if len(splited_element) == 2:
            number_list.append(int(re.sub(",", "", splited_element[1])))
        else:
            number_list.append(0)

    print(number_list)
    max_number = max(number_list)
    click_index = number_list.index(max_number)

    search_list[click_index].click()

In [22]:
input_keyword('오뎅')

[286969, 331321, 57141, 58664, 0, 0, 13202, 25980, 0, 23509, 3283, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2714, 1746, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [72]:
# 인기게시물 9개는 빼고, 최근사진부터 크롤링 시작(인기게시물과 최근게시물에 중복 게시물이 있을거 같아서)
# 처음 렌더링된 화면의 24개의 게시물을 크롤링하는 함수


# 처음 렌더링된 화면의 24개의 게시물을 크롤링하는 함수
def initial_crawling():

    # 검색 키워드를 클릭 한 이후 게시물 렌더링이 완료되었는지 확인
    start_time = time.time()
    result = check_response(
        driver, '#react-root > section > main > article > div:nth-child(3) > div > div', 'initial_crawling', start_time)
    if result == False:
        print('================ initial_crawling rendering error ================')
        driver.quit()

    urls = driver.find_elements_by_xpath(
        '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
    urls = [url.get_attribute("href") for url in urls]

    image_urls = driver.find_elements_by_xpath(
        '//*[@id="react-root"]/section/main/article/div/div/div/div/a/div/div/img')
    image_urls = [url.get_attribute("src") for url in image_urls]

    hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

    for i in range(len(urls)):
        req = requests.get(urls[i])
        response = TextResponse(req.url, body=req.text, encoding="utf-8")
        hash_tag_list = response.css("meta[property='instapp:hashtags']")
        hash_tag_list = ['#' + hash_tag.attrib['content']
                         for hash_tag in hash_tag_list]
        hash_tags = ', '.join(hash_tag_list)

        data = [
            {'time': 0, 'hash_tag': hash_tags, 'image_url': image_urls[i]}
        ]
        df = pd.DataFrame(data)
        hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [73]:
df = initial_crawling()
df

,time,hash_tag,image_url
0,0,"#반찬, #맵자박이, #김미정의요리담다, #제이맘테이블, #은옥상점, #매운어묵볶음...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
1,0,"#당분간은참아보자, #김말이, #돼지런한생활, #가성비갑, #부케데이, #오렌지분식...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
2,0,"#어묵, #튀김, #아이스크림🍦, #김치마요덮밥, #콘치즈떡, #치즈떡, #고구마튀...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
3,0,"#부산오뎅바, #어묵, #오뎅바, #오뎅",https://scontent-ssn1-1.cdninstagram.com/v/t51...
4,0,"#댕댕이그램, #코코, #아기들, #댕댕이스타그램, #오뎅, #댕댕이",https://scontent-ssn1-1.cdninstagram.com/v/t51...
5,0,"#오뎅, #반찬스타그램, #오이무침",https://scontent-ssn1-1.cdninstagram.com/v/t51...
6,0,"#좋은사람, #어제, #먹자골목, #맛있는음식, #오뎅, #감기, #최고의, #수제...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
7,0,"#어묵볶음, #🥕, #banchan, #반찬, #sidedish, #koreanfo...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
8,0,"#프로스트, #인생영화, #여름코디, #피부고민, #좋은아침, #미술놀이, #블라디...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
9,0,"#치즈등갈비, #애월카페, #매콤, #롤링핀, #맛집기행, #핵꿀맛, #점심식사, ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...


In [75]:
df.iloc[5]['image_url']

'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/94397094_224881295464888_1049361863809662757_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=H-w0Qm-TyGcAX8TSyM7&oh=1e4ee279e4da71d7950d6ae7b5de4aec&oe=5ECBF281'

In [102]:
urls = driver.find_elements_by_xpath(
    '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
urls = [url.get_attribute("href") for url in urls]


In [109]:
req = requests.get(urls[2])
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [110]:
pprint.pprint(response.text)

('<!DOCTYPE html>\n'
 '<html lang="en" class="no-js not-logged-in client-root">\n'
 '    <head>\n'
 '        <meta charset="utf-8">\n'
 '        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n'
 '\n'
 '        <title>\n'
 '@yseami on Instagram: “어제 꾹~ ~ 참았던 오뎅국!\n'
 '\n'
 '어찌나 아이들이 맛나게 먹던지 ㅋ ㅋ\n'
 '\n'
 '나도 먹어야지~ \n'
 '#다이어트 #오뎅 #어묵 #아침식사”\n'
 '</title>\n'
 '\n'
 '        \n'
 '        <meta name="robots" content="noimageindex, noarchive">\n'
 '        <meta name="apple-mobile-web-app-status-bar-style" '
 'content="default">\n'
 '        <meta name="mobile-web-app-capable" content="yes">\n'
 '        <meta name="theme-color" content="#ffffff">\n'
 '        <meta id="viewport" name="viewport" content="width=device-width, '
 'initial-scale=1, minimum-scale=1, maximum-scale=1, viewport-fit=cover">\n'
 '        <link rel="manifest" href="/data/manifest.json">\n'
 '\n'
 '        <link rel="preload" '
 'href="/static/bundles/metro/ConsumerUICommons.css/e00dfe54e357.css" '
 'as="style

In [79]:
# 스크롤내리면서 12개씩 추가되는 게시물을 크롤링하는 함수 (두번째만 실행) - chrome 전체화면 기준
# 현재 브라우저 화면 크기에 따라 추가적으로 불러오는 div 갯수가 달라짐
# 전체화면 기준 div의 urls 총 갯수 = 24 -> 36 -> 45 -> 45 -> ...


def second_crawling():

    hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

    driver.execute_script(
        'window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1.5)

    urls = driver.find_elements_by_xpath(
        '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
    urls = [url.get_attribute("href") for url in urls]

    image_urls = driver.find_elements_by_xpath(
        '//*[@id="react-root"]/section/main/article/div/div/div/div/a/div/div/img')
    image_urls = [url.get_attribute("src") for url in image_urls]

    for i in range(len(urls) - 12, len(urls)):
        req = requests.get(urls[i])
        response = TextResponse(
            req.url, body=req.text, encoding="utf-8")
        hash_tag_list = response.css(
            "meta[property='instapp:hashtags']")
        hash_tag_list = ['#' + hash_tag.attrib['content']
                         for hash_tag in hash_tag_list]
        hash_tags = ', '.join(hash_tag_list)

        data = [
            {'time': 0, 'hash_tag': hash_tags, 'image_url': image_urls[i]}
        ]
        df = pd.DataFrame(data)
        hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [80]:
df2 = second_crawling()
df2

,time,hash_tag,image_url
0,0,"#비오는아침, #첫경험, #브뤼셀, #피자빵, #시계줄, #주말운동, #케이뷰티, ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
1,0,"#주말인데, #오뎅, #온카지노, #스냅촬영, #sm, #진행, #휴무, #브뤼셀,...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
2,0,"#instagood, #우동, #좋아요, #instafood, #맞팔, #신혼부부,...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
3,0,"#중봉세끼🍚, #야식, #일상, #오뎅, #데일리, #인스타, #사진, #순대, #...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
4,0,"#포장마차떡볶이, #떡볶이, #옥수역떡볶이, #순대, #여기가맛집, #20년단골집👍...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
5,0,"#노래가절로, #짜장면, #모짜렐라소세지핫도그, #엄마밥도잘먹어주라, #아니었음, ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
6,0,"#애니, #오뎅, #아시안카지노, #새해, #육아맘일상, #좋아요꾹, #행복한하루,...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
7,0,"#foodstagram, #pizza, #ddeokbokki, #순대, #순대모듬볶...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
8,0,"#네임드사다리, #식스팩, #땅끝마을, #여행가고싶다, #여행, #아빠, #야구, ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
9,0,"#야한섹스동영상, #사카시, #맞선, #이벤트속옷, #해바라기, #오뎅, #천호, ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...


In [98]:
print(df2['image_url'][4])

https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/94129958_327191658244370_5689437216176539816_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=qy7AJA7IGccAX_xAJza&oh=1e695c1684cdba08840be278ebf3dd3f&oe=5ECB32EF


In [99]:
df2['hash_tag'][4]

'#포장마차떡볶이, #떡볶이, #옥수역떡볶이, #순대, #여기가맛집, #20년단골집👍, #오뎅, #어묵, #튀김'

In [85]:
# 세번째부터는 이 함수 반복하기 -  게시물 9개씩 크롤링


def repeat_crawling():

    hash_tag_df = pd.DataFrame(columns=['time', 'hash_tag'])

    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1)

    urls = driver.find_elements_by_xpath(
        '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
    urls = [url.get_attribute("href") for url in urls]

    image_urls = driver.find_elements_by_xpath(
        '//*[@id="react-root"]/section/main/article/div/div/div/div/a/div/div/img')
    image_urls = [url.get_attribute("src") for url in image_urls]

    for i in range(len(urls) - 9, len(urls)):
        req = requests.get(urls[i])
        response = TextResponse(req.url, body=req.text, encoding="utf-8")
        hash_tag_list = response.css("meta[property='instapp:hashtags']")
        hash_tag_list = ['#' + hash_tag.attrib['content']
                         for hash_tag in hash_tag_list]
        hash_tags = ', '.join(hash_tag_list)

        data = [
            {'time': 0, 'hash_tag': hash_tags, 'image_url': image_urls[i]}
        ]
        df = pd.DataFrame(data)
        hash_tag_df = hash_tag_df.append(df)

    return hash_tag_df.reset_index().drop(columns=['index'])

In [87]:
df3 = repeat_crawling()
df3

,time,hash_tag,image_url
0,0,"#한국요리, #추억, #혼자여행, #김밥, #맛집, #おでん, #여행스타그램, #먹...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
1,0,"#성내동, #분식, #강동구, #둔촌동, #첫줄, #떡볶이, #붕어빵, #오뎅, #...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
2,0,"#강남야구장, #출장안마, #오뎅, #소주안주, #다이어트중, #오사카성, #노출증...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
3,0,,https://scontent-ssn1-1.cdninstagram.com/v/t51...
4,0,"#마시쪙💕, #고양이, #핫바, #바다, #고양이타투, #오뎅, #산책스타그램, #...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
5,0,"#오골계, #코숏, #스코티시폴드, #똥꼬발랄, #집사스타그램, #냐옹이그램, #스...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
6,0,"#소사벌, #평택맛집, #맛집, #반가의소반, #견과류쌈장, #가리비젓갈, #다래순...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
7,0,"#분식이이래도되는가, #만두, #떡볶이, #비빔면도, #꿀맛, #오뎅, #음식그램✌...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
8,0,"#하동의아들, #❣️, #추운날엔, #쫌먹을줄아네, #먹방, #쫌먹을줄아는너, #❤...",https://scontent-ssn1-1.cdninstagram.com/v/t51...


In [93]:
print(df3['image_url'][4])

https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/sh0.08/e35/s640x640/94196292_563825107863861_1613987456354405743_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=109&_nc_ohc=vdCYxsNXinQAX90KXok&oh=b1a7f7531f1dd07aea2b6c7698dcd617&oe=5ECA9A7D


In [94]:
df3['hash_tag'][4]

'#마시쪙💕, #고양이, #핫바, #바다, #고양이타투, #오뎅, #산책스타그램, #산책중, #오징어, #오뎅탕, #고양이스타그램, #산, #생선구이맛집, #😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎😎'

In [95]:
df2

,time,hash_tag,image_url
0,0,"#비오는아침, #첫경험, #브뤼셀, #피자빵, #시계줄, #주말운동, #케이뷰티, ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
1,0,"#주말인데, #오뎅, #온카지노, #스냅촬영, #sm, #진행, #휴무, #브뤼셀,...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
2,0,"#instagood, #우동, #좋아요, #instafood, #맞팔, #신혼부부,...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
3,0,"#중봉세끼🍚, #야식, #일상, #오뎅, #데일리, #인스타, #사진, #순대, #...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
4,0,"#포장마차떡볶이, #떡볶이, #옥수역떡볶이, #순대, #여기가맛집, #20년단골집👍...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
5,0,"#노래가절로, #짜장면, #모짜렐라소세지핫도그, #엄마밥도잘먹어주라, #아니었음, ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
6,0,"#애니, #오뎅, #아시안카지노, #새해, #육아맘일상, #좋아요꾹, #행복한하루,...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
7,0,"#foodstagram, #pizza, #ddeokbokki, #순대, #순대모듬볶...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
8,0,"#네임드사다리, #식스팩, #땅끝마을, #여행가고싶다, #여행, #아빠, #야구, ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...
9,0,"#야한섹스동영상, #사카시, #맞선, #이벤트속옷, #해바라기, #오뎅, #천호, ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...


In [21]:
import pprint

In [17]:
# 맨처음 화면의 게시물 갯수

urls = driver.find_elements_by_xpath(
    '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
urls = [url.get_attribute("href") for url in urls]
len(urls)

24

In [577]:
# 스크롤을 한번 끝까지 내렸을때 새로 불러온 게시물을 포함한 전체 게시물 갯수

driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
time.sleep(1.5)
urls = driver.find_elements_by_xpath(
    '//*[@id="react-root"]/section/main/article/div[2]/div/div/div/a')
urls = [url.get_attribute("href") for url in urls]
len(urls)

45

In [18]:
req = requests.get(urls[0])
req.url

'https://www.instagram.com/p/B9pxuKenKyG/'

In [19]:
response = TextResponse(req.url, body=req.text, encoding="utf-8")

response

<200 https://www.instagram.com/p/B9pxuKenKyG/>

In [78]:
# pprint.pprint(response.text)

In [581]:
response.xpath(
    '//*[@id="react-root"]/section/main/div/div[1]/article/div[2]/div[1]/ul/div/li/div/div/div[2]/span/a')

[]

In [79]:
# pprint.pprint(req.text)

In [32]:
driver.quit()